In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/translated-data-nepali/translated_data_nepali.tsv
/kaggle/input/liar-fake-news-dataset/test.tsv
/kaggle/input/liar-fake-news-dataset/README
/kaggle/input/liar-fake-news-dataset/train.tsv
/kaggle/input/liar-fake-news-dataset/valid.tsv


In [5]:
!pip install inflect
!pip install num2words 
!pip install deep_translator

In [6]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt 
from deep_translator import GoogleTranslator
import inflect
from num2words import num2words

In [35]:
df = pd.read_csv('/kaggle/input/liar-fake-news-dataset/train.tsv', sep='\t')

In [36]:
df.head()

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [37]:
df.shape

(10239, 14)

In [38]:
for col in df.columns:
    print(col)

2635.json
false
Says the Annies List political group supports third-trimester abortions on demand.
abortion
dwayne-bohac
State representative
Texas
republican
0
1
0.1
0.2
0.3
a mailer


In [39]:
df.columns=['id', 'label', 'statement', 'subjects','speaker', 'speaker_job_title', 'state_info', 'party_affilation',
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement) 
           ]
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [40]:
df.value_counts('label')

label
half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
dtype: int64

In [ ]:
import string
def removepunctuation(sentence):
   new_text = sentence.replace("—", " ") # "—" in the text was not equivalent to "-" so we replcaed "—" with a space
   print 
  #iterate over every single character to see if it is a punctuation or not, and then concatenate them using "join"
   punctuationfree="".join([i for i in new_text if i not in string.punctuation])
   return punctuationfree


In [52]:
df["statement_removedpunctuation"] = df['statement'].apply(lambda x : removepunctuation(x))

In [53]:
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,When did the decline of coal start It started ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agrees with John McCain by vot...
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,Health care reform legislation is likely to ma...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,The economic turnaround started at the end of ...
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,The Chicago Bears have had more starting quart...


In [54]:
df['statement']

0        When did the decline of coal start? It started...
1        Hillary Clinton agrees with John McCain "by vo...
2        Health care reform legislation is likely to ma...
3        The economic turnaround started at the end of ...
4        The Chicago Bears have had more starting quart...
                               ...                        
10234    There are a larger number of shark attacks in ...
10235    Democrats have now become the party of the [At...
10236    Says an alternative to Social Security that op...
10237    On lifting the U.S. Cuban embargo and allowing...
10238    The Department of Veterans Affairs has a manua...
Name: statement, Length: 10239, dtype: object

In [56]:
df['statement_removedpunctuation'][7]

'However it took 195 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations CenterPacific'

In [ ]:
translated = GoogleTranslator(source='auto', target='ne').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig
translated

In [ ]:
random =  df.sample(10)
random['statement'].iloc[0]


In [ ]:
translated = GoogleTranslator(source='auto', target='ne').translate(random['statement'].iloc[0])  # output -> Weiter so, du bist großartig
translated

In [ ]:
random['translated_value'] = random['statement'].apply(lambda x: GoogleTranslator(source='auto', target='ne').translate(x))

In [ ]:
random['translated_value']

In [ ]:
random['retranslated_value'] = random['translated_value'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))


In [ ]:
random['retranslated_value'].iloc[0]

In [ ]:
random['statement'].iloc[2]

In [ ]:
index=2
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

In [ ]:
index=3
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

In [57]:
new_df = df.copy()
new_df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,When did the decline of coal start It started ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agrees with John McCain by vot...
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,Health care reform legislation is likely to ma...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,The economic turnaround started at the end of ...
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,The Chicago Bears have had more starting quart...


In [59]:
def translate_to_nepali(text, target_lang):
    # Convert numbers to text using the inflect library
    p = inflect.engine()
    words = text.split()
    for i, word in enumerate(words):
        if word.isnumeric():
            words[i] = p.number_to_words(word)
    text = ' '.join(words)
    
    # Translate the text using GoogleTranslator
    translated_text = GoogleTranslator(source='auto', target=target_lang).translate(text)
    
    return translated_text

In [60]:
new_df['nepali_translation'] = new_df['statement_removedpunctuation'].apply(lambda x: translate_to_nepali(x,"ne"))


In [63]:
new_df["nepali_translation"][7]

'यद्यपि यसले न्यूपोर्टको पोर्टको लागि ओरेगन लटरी कोषमा एक सय पचानब्बे मिलियन लियो अन्ततः नयाँ NOAA मरीन अपरेशन्स सेन्टर प्यासिफिकमा अवतरण गर्न।'

In [61]:
new_df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation,nepali_translation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,When did the decline of coal start It started ...,कोइलाको पतन कहिले सुरु भयो यो प्राकृतिक ग्यास ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agrees with John McCain by vot...,हिलारी क्लिन्टनले जर्ज बुशलाई इरानमा शंकाको फा...
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,Health care reform legislation is likely to ma...,स्वास्थ्य हेरचाह सुधार कानूनले नि: शुल्क लिंग ...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,The economic turnaround started at the end of ...,मेरो कार्यकालको अन्त्यमा आर्थिक परिवर्तन सुरु भयो
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,The Chicago Bears have had more starting quart...,शिकागो बियर्सले विगत दुई दशकहरूमा बर्खास्त गरि...


In [62]:
new_df.to_csv('/kaggle/working/translated_data_nepali1.tsv', sep='\t', index=False)


In [65]:
df1 = pd.read_csv('/kaggle/input/translated-data-nepali1/translated_data_nepali1.tsv', sep='\t')

In [66]:
df1['nepali_translation'][7]

'यद्यपि यसले न्यूपोर्टको पोर्टको लागि ओरेगन लटरी कोषमा एक सय पचानब्बे मिलियन लियो अन्ततः नयाँ NOAA मरीन अपरेशन्स सेन्टर प्यासिफिकमा अवतरण गर्न।'

In [67]:
df1.shape

(10239, 16)

In [10]:
new_df1 = df1.copy()
new_df1['nepali_translation'].iloc(0)

In [11]:
new_df1['nepali_translation'][10]

'2000 यता, लगभग 12 मिलियन अमेरिकीहरू मध्यम वर्ग र गरिबीमा फसेका छन्।'

In [12]:


def translate_from_nepali_english(text, target_lang, reverse=False):
    if not reverse:
        # Split the sentence into words
        words = text.split()

        # Convert any numeric words to text using the num2words library
        for i, word in enumerate(words):
            if word.isnumeric():
                words[i] = num2words(int(word), lang='en')

        # Join the words back into a sentence and translate it using GoogleTranslator
        modified_text = ' '.join(words)
        translated_text = GoogleTranslator(source='auto', target=target_lang).translate(modified_text)

        return translated_text
    else:
        # Translate the text back to the original language
        translated_text = GoogleTranslator(source=target_lang, target='en').translate(text)

        # Replace any numeric text with their corresponding numerical values
        words = translated_text.split()
        for i, word in enumerate(words):
            try:
                numeric_value = int(num2words(word, lang='en'))
                words[i] = str(numeric_value)
            except ValueError:
                pass
        # Join the words back into a sentence and return it
        modified_text = ' '.join(words)
        return modified_text

# translate_from_nepali_english('अमेरिकी जनगणना ब्यूरोका अनुसार, टेक्सासमा 71.2 प्रतिशत हिस्पैनिक दर्ता मतदाता र 86 प्रतिशत भन्दा बढी अफ्रिकी अमेरिकी दर्ता मतदाताहरूले 2012 को चुनावमा भाग लिए।' , 'en')


In [15]:

def translate_to_english(text, target_lang):
    # Convert numbers to text using the inflect library
    p = inflect.engine()
    words = text.split()
    for i, word in enumerate(words):
        if word.isnumeric():
            words[i] = p.number_to_words(word)
    text = ' '.join(words)
    
    # Translate the text using GoogleTranslator
    translated_text = GoogleTranslator(source='auto', target=target_lang).translate(text)
    
    return translated_text



In [34]:
translate_to_english(df1['nepali_translation'][7], "en")


'However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to finally land the new NOAA Marine Operations Center-Pacific.'

In [68]:
new_df1['english_translation'] = new_df1['nepali_translation'].apply(lambda x:GoogleTranslator(source='auto', target=target_lang).translate(text)
)


translate_to_english(new_df1['nepali_translation'][200],"en")


In [ ]:
new_df.head()

In [ ]:
new_df.to_csv('/kaggle/working/translated_data_english.tsv', sep='\t', index=False)
